In [6]:
import pandas as pd
import numpy as np
#import wget #WGET doesn't work here: URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1125)>
import requests
import psycopg2
from datetime import date, datetime, timedelta, timezone


In [7]:
directory =  "..\..\data\sensorCommunity\\"

yesterday = (date.today() - timedelta(days=1)).strftime("%Y-%m-%d")
sensortype = "sds011"
sensorid = "60362" #"60368"

print(yesterday)

2021-10-18


# Code

In [8]:
import io,csv
from typing import List,Iterable

In [9]:
url = "https://archive.sensor.community/%s/%s_%s_sensor_%s.csv" % (str(yesterday),str(yesterday),sensortype,sensorid)
print(url)

https://archive.sensor.community/2021-10-18/2021-10-18_sds011_sensor_60362.csv


In [16]:
res = requests.get(url, stream=True)
if res.ok:
    file_ = io.BytesIO(res.content)
else:
     raise IOError()

In [17]:
buffer = io.StringIO(file_.read().decode('UTF-8'))

In [18]:
class ZepyhrSensor:
    """Per sensor object designed to wrap the csv files returned by the Plume API.

    Example Usage:
        ps = PlumeSensor.from_csv("16397", open("sensor_measures_20211004_20211008_1.csv"))
        print(ps.DataFrame)
    """

    def __init__(self, id_, header: List, rows: List):
        self.id = id_
        self.header = header
        self.rows = rows

    def add_row(self, row: Iterable):
        """Normalise and append row to internal list.

        Coverts all digits to int objects, all elements are initially converted to strings before
        digit check to avoid type errors.

        :param row: row to add to plume sensor
        """
        self.rows.append([int(i) if str(i).isdigit() else i for i in row])
    
    @property
    def dataframe(self) -> pd.DataFrame:
        """Writes headers and rows into a DataFrame.
        """
        return pd.DataFrame(self.rows, columns=self.header)

In [19]:
sensor_id = '814'
reader = csv.reader(buffer, dialect=csv.unix_dialect, delimiter = ';')
header = next(reader)
sensor = ZepyhrSensor(sensor_id, header, [])
for row in reader:
    sensor.add_row(row)

In [20]:
sensor.dataframe.head()

,sensor_id,sensor_type,location,lat,lon,timestamp,P1,durP1,ratioP1,P2,durP2,ratioP2
0,60362,SDS011,46205,48.870,12.562,2021-10-18T00:00:15,61.50,,,34.78,,
1,60362,SDS011,46205,48.870,12.562,2021-10-18T00:02:42,66.40,,,36.90,,
2,60362,SDS011,46205,48.870,12.562,2021-10-18T00:05:09,70.15,,,37.40,,
3,60362,SDS011,46205,48.870,12.562,2021-10-18T00:07:38,66.53,,,34.58,,
4,60362,SDS011,46205,48.870,12.562,2021-10-18T00:10:05,59.42,,,34.70,,


# Code

In [3]:
url = "https://archive.sensor.community/%s/%s_%s_sensor_%s.csv" % (str(yesterday),str(yesterday),sensortype,sensorid)
print(url)

https://archive.sensor.community/2021-10-14/2021-10-14_sds011_sensor_60362.csv


In [4]:
# Define the local filename to save data
local_file = directory + url.split('_')[-1]
# Make http request for remote file data
data = requests.get(url)
# Save file data to local copy
with open(local_file, 'wb')as file:
    file.write(data.content)

In [5]:
sensorDirectory = next(os.walk(directory))[2]
print(sensorDirectory)

['60362.csv', '60368.csv']


In [6]:
#extracts all the sensorIDs from the list of sensor directories and adds it to a list
sensorIds = []

for currentId in sensorDirectory:
    sensorIds.append(currentId.split(".")[0])
print(sensorIds)

['60362', '60368']


In [7]:
sensorPath = []
for i in sensorIds:
    sensorPath.append(os.path.join(directory + i + ".csv"))

#map serial number to filepaths
sensorPaths = {k:v for k,v in zip(sensorIds,sensorPath)}
print(sensorPaths)

{'60362': '..\\..\\data\\sensorCommunity\\60362.csv', '60368': '..\\..\\data\\sensorCommunity\\60368.csv'}


# Extract csv to dataframes

In [15]:
for key in sensorPaths:
    df = pd.read_csv(sensorPaths[key],sep=';', parse_dates=True, index_col= "timestamp")
    df.drop("sensor_id",axis=1,inplace=True)
    df.drop("sensor_type",axis=1,inplace=True)
    df.drop("location",axis=1,inplace=True)
    break

In [16]:
df.head()

,sensor_type,lat,lon,P1,durP1,ratioP1,P2,durP2,ratioP2
timestamp,,,,,,,,,
2021-10-14 00:00:16,SDS011,48.87,12.562,13.85,NaN,NaN,7.90,NaN,NaN
2021-10-14 00:02:42,SDS011,48.87,12.562,15.32,NaN,NaN,8.57,NaN,NaN
2021-10-14 00:05:09,SDS011,48.87,12.562,13.80,NaN,NaN,8.10,NaN,NaN
2021-10-14 00:07:35,SDS011,48.87,12.562,12.95,NaN,NaN,8.60,NaN,NaN
2021-10-14 00:10:03,SDS011,48.87,12.562,15.43,NaN,NaN,8.65,NaN,NaN
